In [1]:
!pip3 install --upgrade --quiet langchain langchain-community langchain-openai chromadb
!pip3 install --upgrade --quiet pypdf pandas streamlit python-dotenv
!pip3 install langchain-chroma


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [2]:
# Import langchain modules
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
# from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
# from pydantic import BaseModel, ImportString, PydanticUserError, Field
from langchain_chroma import Chroma

# other modules and packages
import os
import tempfile
import streamlit as st
import pandas as pd
import numpy
from dotenv import load_dotenv

/Users/sgnclexus/Development/rag_llms/myenv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
load_dotenv()

True

In [4]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [5]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)
llm.invoke("Tell me one example of how to achieve to read an understand a book?")

AIMessage(content="One effective method to read and understand a book is the **SQ3R** technique, which stands for Survey, Question, Read, Recite, and Review. Here's how you can use it:\n\n1. **Survey**: Start by skimming the book. Look at the table of contents, chapter titles, headings, subheadings, and any summaries or highlighted text. This gives you an overview of the main ideas and structure.\n\n2. **Question**: Formulate questions based on your survey. What do you want to learn from the book? This could involve specific topics or themes you’re interested in.\n\n3. **Read**: Read the book actively, focusing on finding answers to your questions. Take notes, highlight important passages, and jot down any thoughts or reactions as you go.\n\n4. **Recite**: After completing a section or chapter, pause to summarize what you've read in your own words. This can be done verbally or in writing. It helps reinforce your understanding and memory of the material.\n\n5. **Review**: Finally, go ba

### Load PDF Document

In [6]:
loader = PyPDFLoader("data/CONEVAL Derecho a la educacion (EDDE_2024_IF).pdf")
pages = loader.load()
pages 

[Document(metadata={'source': 'data/CONEVAL Derecho a la educacion (EDDE_2024_IF).pdf', 'page': 0, 'page_label': '1'}, page_content=' \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n '),
 Document(metadata={'source': 'data/CONEVAL Derecho a la educacion (EDDE_2024_IF).pdf', 'page': 1, 'page_label': '2'}, page_content=' \n2 \n \nCONSEJO ACADÉMICO SECRETARÍA EJECUTIVA \nArmando Bartra Vergés \nUniversidad Autónoma Metropolitana \nXochimilco \nJosé Nabor Cruz Marcelo \nSecretario Ejecutivo \nMaría del Rosario Cárdenas Elizalde \nUniversidad Autónoma Metropolitana \nXochimilco \nKarina Barrios Sánchez \nCoordinadora General de Evaluación \nGuillermo Miguel Cejudo Ramírez \nCentro de Investigación y Docencia Económicas \nAlida Marcela Gutiérrez Landeros \nCoordinadora General de Análisis de la Pobreza \nClaudia Vanessa Maldonado Trujillo \nUniversidad Autónoma Metropolitana \nXochimilco \nJosé Manuel Del Muro Guerrero \nCoordinador General de Monitoreo, En

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                               chunk_overlap=500,
                                               length_function=len,
                                               separators=["\n\n","\n"," "])
chunks = text_splitter.split_documents(pages)
chunks

[Document(metadata={'source': 'data/CONEVAL Derecho a la educacion (EDDE_2024_IF).pdf', 'page': 1, 'page_label': '2'}, page_content='2 \n \nCONSEJO ACADÉMICO SECRETARÍA EJECUTIVA \nArmando Bartra Vergés \nUniversidad Autónoma Metropolitana \nXochimilco \nJosé Nabor Cruz Marcelo \nSecretario Ejecutivo \nMaría del Rosario Cárdenas Elizalde \nUniversidad Autónoma Metropolitana \nXochimilco \nKarina Barrios Sánchez \nCoordinadora General de Evaluación \nGuillermo Miguel Cejudo Ramírez \nCentro de Investigación y Docencia Económicas \nAlida Marcela Gutiérrez Landeros \nCoordinadora General de Análisis de la Pobreza \nClaudia Vanessa Maldonado Trujillo \nUniversidad Autónoma Metropolitana \nXochimilco \nJosé Manuel Del Muro Guerrero \nCoordinador General de Monitoreo, Entidades \nFederativas y Fortalecimiento Institucional \nSalomón Nahmad Sittón \nCentro de Investigaciones y Estudios \nSuperiores en Antropología Social-Pacífico Sur \nDaniel Gutiérrez Cruz \nCoordinador General de Administra

### Create embeddings

In [10]:
def get_embedding_function():
    embeddings = OpenAIEmbeddings(
        model="text-embedding-ada-002", openai_api_type=OPENAI_API_KEY
    )
    return embeddings

embedding_function = get_embedding_function()
test_vector = embedding_function.embed_query("dog")

In [11]:
from langchain.evaluation import load_evaluator

evaluator = load_evaluator(evaluator="embedding_distance",
                           embeddings=embedding_function)

evaluator.evaluate_strings(prediction="Amsterdam", reference="coffeeshop")

{'score': 0.1745443723078154}

### Create a vector database

In [12]:
import uuid

def create_vectorstore(chunks, embedding_function, vectorstore_path):

    # create a list of unique ids for each document based on the content
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]

    # ensure that only unique docs with unique ids are kept
    unique_ids = set()
    unique_chunks = []
    
    unique_chunks = []
    for chunk, id in zip(chunks, ids):
        if id not in unique_ids:
            unique_ids.add(id)
            unique_chunks.append(chunk)


    # Create a new Chroma database from the documents
    vectorstore = Chroma.from_documents(documents=chunks,
                                        ids=list(unique_ids),
                                        embedding=embedding_function,
                                        persist_directory="vectorstore_edu")

    # vectorstore.persist()
    # vectorstore._persist_directory()

    return vectorstore

In [13]:
 # create vectorstore
vectorstore = create_vectorstore(chunks=chunks,
                                 embedding_function=embedding_function,
                                 vectorstore_path="vectorstore_edu"
                                 )

 ## 2. Query for relevant data

In [14]:
# load vectorstore
vectorstore = Chroma(persist_directory="vectorstore_edu", embedding_function=embedding_function)

In [15]:
# create retriever and get relevant chunks
retriever = vectorstore.as_retriever(search_type="similarity")
relevant_chunks = retriever.invoke("Por favor muestrame los puntos mas importantes del documento")
relevant_chunks

[Document(id='5128b33f-6c4d-5870-ba1d-d7df589fffa4', metadata={'page': 12, 'page_label': '13', 'source': 'data/CONEVAL Derecho a la educacion (EDDE_2024_IF).pdf'}, page_content='observados en cada aspecto del derecho, con lo cual se busca identificar las problemáticas \nmás relevantes que pueden estar asociadas a esos resultados, así como las poblaciones \nque se encuentran en mayor desventaja.  \nCon base en los resultados de ese análisis diagnóstico, en el tercer apartado se presentan \nlas principales brechas o poblaciones cuyo ejercicio del derecho se encuentra \nsistemáticamente comprometido, y finalmente se identifican y presentan de manera \nsintética los principales retos que enfrenta la política social para avanzar en la garantía de \nlos derechos.'),
 Document(id='e2389534-3e6c-5817-b3c6-1598cb225bf3', metadata={'page': 193, 'page_label': '194', 'source': 'data/CONEVAL Derecho a la educacion (EDDE_2024_IF).pdf'}, page_content='cuanto a su infraestructura y disponibilidad de m

In [16]:
PROMPT_TEMPLATE = """
    Trabajas en el ramo de la educacion y te interesa saber los puntos 
    mas relevantes del documento que envio la secretaria de educacion publica. 
    

    {context}

    ---

    Responde las preguntas de acuerdo al contexto proporcionado: {question}
"""



In [17]:
# concatenate context text
context_text = "\n\n---\n\n".join([doc.page_content for doc in relevant_chunks])

# create prompt
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question="Por favor muestrame los puntos mas importantes del documento")
print(prompt)

Human: 
    Trabajas en el ramo de la educacion y te interesa saber los puntos 
    mas relevantes del documento que envio la secretaria de educacion publica. 
    

    observados en cada aspecto del derecho, con lo cual se busca identificar las problemáticas 
más relevantes que pueden estar asociadas a esos resultados, así como las poblaciones 
que se encuentran en mayor desventaja.  
Con base en los resultados de ese análisis diagnóstico, en el tercer apartado se presentan 
las principales brechas o poblaciones cuyo ejercicio del derecho se encuentra 
sistemáticamente comprometido, y finalmente se identifican y presentan de manera 
sintética los principales retos que enfrenta la política social para avanzar en la garantía de 
los derechos.

---

cuanto a su infraestructura y disponibilidad de materiales educativos, la calidad docente y 
las prácticas de enseñanza- aprendizaje (Treviño, y otros, 2010). Por tanto, la literatura 
 
72 Planea evalúa el logro de aprendizajes clave de los

In [18]:
llm.invoke(prompt)

AIMessage(content='Claro, a continuación te presento los puntos más relevantes del documento relacionado con la educación, que se aportan en los fragmentos que compartiste:\n\n1. **Análisis Diagnóstico**: Se realiza un análisis para identificar problemáticas asociadas a la garantía del derecho a la educación, enfocándose en aquellas poblaciones en desventaja. Se busca destacar las brechas que afectan el ejercicio de este derecho.\n\n2. **Retos de la Política Social**: Se identifican retos cruciales que enfrenta la política social para asegurar que los derechos educativos sean garantizados, enfatizando la necesidad de avanzar en este ámbito.\n\n3. **Calidad de la Educación**: Se enfoca en varios factores que afectan el rendimiento escolar de los estudiantes, como:\n   - **Infraestructura y Materiales**: La calidad de las instituciones educativas en términos de infraestructura y recursos disponibles.\n   - **Calidad Docente**: La formación y desempeño de los docentes en el proceso de ens

In [ ]:
AIMessage(content='Claro, a continuación te presento los puntos más relevantes del documento relacionado con la educación, que se aportan en los fragmentos que compartiste:\n\n1. **Análisis Diagnóstico**: Se realiza un análisis para identificar problemáticas asociadas a la garantía del derecho a la educación, enfocándose en aquellas poblaciones en desventaja. Se busca destacar las brechas que afectan el ejercicio de este derecho.\n\n2. **Retos de la Política Social**: Se identifican retos cruciales que enfrenta la política social para asegurar que los derechos educativos sean garantizados, enfatizando la necesidad de avanzar en este ámbito.\n\n3. **Calidad de la Educación**: Se enfoca en varios factores que afectan el rendimiento escolar de los estudiantes, como:\n   - **Infraestructura y Materiales**: La calidad de las instituciones educativas en términos de infraestructura y recursos disponibles.\n   - **Calidad Docente**: La formación y desempeño de los docentes en el proceso de enseñanza-aprendizaje.\n   - **Prácticas Pedagógicas**: La efectividad de las metodologías empleadas para impartir educación.\n\n4. **Evaluación de Aprendizajes**: El documento menciona la importancia de evaluar el logro de aprendizajes clave en áreas críticas como el lenguaje y la comunicación, y matemáticas, así como en otras áreas del currículo educativo.\n\n5. **Retroalimentación a Docentes**: Establece que debe proporcionarse retroalimentación oportuna a los docentes respecto al desempeño de sus alumnos para que puedan realizar ajustes didácticos adecuados.\n\n6. **Contexto Familiar y Personal**: Se reconoce que las características personales y familiares de los estudiantes, especialmente las condiciones socioeconómicas, impactan directamente en su rendimiento escolar.\n\n7. **Desigualdad Educativa**: En el capítulo sobre brechas de desigualdad, se analizan las diferencias en el acceso y ejercicio del derecho a la educación por grupos etarios, incluyendo categorías como la primera infancia (0-2 años), niños y niñas de 3 a 5 años, y adolescentes de 15 a 17 años.\n\nEstos puntos reflejan un análisis profundo sobre la situación actual de la educación en relación a los derechos, las brechas existentes y los factores que influyen en el desempeño y acceso a la educación.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 468, 'prompt_tokens': 752, 'total_tokens': 1220, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-36b76fb2-4092-4190-be2f-355db6b794cb-0', usage_metadata={'input_tokens': 752, 'output_tokens': 468, 'total_tokens': 1220, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})